# OpenUVF Image Segmentation

Copyright © 2019 Southern Company Services, Inc.  All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
#Import Statements
import cv2 as cv
import os
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import math
import lensfunpy
import core.utils.segmentation_utils as segment
from random import sample
from PIL import Image as im


### Inputs

#### Directories
1. images_dir = string specifiying the path (local or global) to the image directory


#### Module parameters
1. module_orientation = string specifying the orientation of the module. Choices are 'vertical' or ' horizontal' 
2. module_aspect_ratio = float specifying the aspect ratio of the module, specified as ratio of longer side to shorter side


#### Function Settings
1. debug = boolean specifying if debug plots should be generated
2. itmax = int specifying the maximum amount of refinement iterations where the segmentation routine is adjusted to increase accuracy

In [2]:
# Directories
images_dir = 'workspace\module_segmentation\Sample Set\FullSize'
images_ext = '.JPG'
output_dir = 'workspace\module_segmentation\Sample Set\Processed/'

# Imaging Parameters 
num_rows = 1
module_orientation = 'horizontal'
module_aspect_ratio = 2

# Camera Distortion Parameters
correct_distortion = True
correct_distortion_mode = 'lensfun'
camera_mnfcr = 'NIKON CORPORATION'
camera_model = 'NIKON D3S'
lens_mnfcr = 'SAMYANG'
lens_model = 'Samyang T-S 24mm f/3.5 ED AS UMC'
image_focal_length = 24.0
image_aperture = 1.4
approx_distance = 3
camera_calibration = None


# Function Settings
debug = True
debug_outputs = True
itmax = 10

### Default Plotting Parameters

In [3]:
mpl.rcParams.keys()
mpl.rcParams['axes.spines.bottom'] = False
mpl.rcParams['axes.spines.left'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False

### Debug Plots

In [4]:
def generateDebugPlots():
        
    #Create figure
    fig = plt.figure(num=f'Image {image_name} Preprocessing', tight_layout=True,figsize=[18, 9.5])
    gs = gridspec.GridSpec(4,5)
    mng = plt.get_current_fig_manager()

    # Axis RGB - Plot base image
    axrgb = fig.add_subplot(gs[0,0])
    plt.imshow(rgb)
    plt.title('RGB')

    # Axis R - Plot base image
    axr = fig.add_subplot(gs[0,1])
    plt.imshow(red) 
    plt.title('Red')

    # Axis G - Plot base image
    axg = fig.add_subplot(gs[0,2])
    plt.imshow(gre) 
    plt.title('Green')

    # Axis B - Plot base image
    axb = fig.add_subplot(gs[0,3])
    plt.imshow(blu) 
    plt.title('Blue')

    # Axis RGB Hist - Plot histograms of the three channels
    axrgbhist = fig.add_subplot(gs[0,4])
    plt.plot(red_hist, 'r-'), plt.plot(gre_hist, 'g-'), plt.plot(blu_hist, 'b-')
    plt.xlabel('Intensity'), plt.ylabel('Count'), plt.title('RGB Histograms')

    # Axis HSV - Plot image in hsv color space
    axhsv = fig.add_subplot(gs[1,0])
    plt.imshow(hsv)
    plt.title('HSV')

    # Axis H - Plot image in hsv color space
    axh = fig.add_subplot(gs[1,1])
    plt.imshow(hue)
    plt.title('Hue')

    # Axis S - Plot image in hsv color space
    axs = fig.add_subplot(gs[1,2])
    plt.imshow(sat)
    plt.title('Saturation')

    # Axis V - Plot image in hsv color space
    axv = fig.add_subplot(gs[1,3])
    plt.imshow(val)
    plt.title('Value (brightness)')

    # Axis Hist - Plot histograms of the three channels
    axhsvhist = fig.add_subplot(gs[1,4])
    plt.plot(hue_hist, 'k-', label = 'Hue')
    plt.plot(sat_hist, 'm-', label = 'Sat')
    plt.plot(val_hist, 'c-', label = 'Val')
    plt.xlabel('Intensity'), plt.ylabel('Count'), plt.title('HSV Histograms'), plt.legend(loc='upper left')

    # Axis Grau - Plot standard grayscale image
    axgray = fig.add_subplot(gs[2,0])
    plt.imshow(gray)
    plt.title('Grayscale')

    # Axis Gray Histogram
    axgrayhist = fig.add_subplot(gs[2,1])
    gray_hist = cv.calcHist([gray], [0], None, [256], [0, 256])
    plt.plot(gray_hist, 'k-')
    plt.title('Grayscale Histogram')

    # Axis RGB min - Plot standard grayscale image
    axgray = fig.add_subplot(gs[2,2])
    plt.imshow(rgb_min)
    plt.title('RGB Min (min of each channel)')

    # Axis RGB min Histogram
    axgrayhist = fig.add_subplot(gs[2,3])
    rgb_min_hist = cv.calcHist([rgb_min], [0], None, [256], [0, 256])
    plt.plot(rgb_min_hist, 'k-')
    plt.title('RGB min Histogram')

    # Axis RGB min edges 
    axhe = fig.add_subplot(gs[3,0])
    plt.imshow(edges_rgb_min)
    plt.title('RGB Min Edges')

    # Axis Gray edges - Plot showing the edges detected in the standard grayscale image
    axhe = fig.add_subplot(gs[3,1])
    plt.imshow(edges_gray)
    plt.title('Grayscale Edges')

    # Axis Hue edges - Plot showing the edge
    axhe = fig.add_subplot(gs[3,2])
    plt.imshow(edges_hue)
    plt.title('Hue Edges')

    # Axis final edges - Plot showing the edge
    axhe = fig.add_subplot(gs[3,3])
    plt.imshow(edges)
    plt.title('Edges')

    # Axis Hue Lines - Plot detected lines 
    axhe = fig.add_subplot(gs[3,4])
    plt.imshow(gray_bgr)
    plt.title('Detected Lines (Hough)')

    #Create Second figure
    fig2 = plt.figure(num=f'Image {image_name} Segmentation', tight_layout=True,figsize=[18, 9.5])
    gs2 = gridspec.GridSpec(2,3)
    mng2 = plt.get_current_fig_manager()

    #Selected representative perspective lines
    axls = fig2.add_subplot(gs2[0,0])
    plt.imshow(gray_sample_lines)
    plt.title('Projective Transform Sample Lines')

    #Vanishing point plot
    axvps = fig2.add_subplot(gs2[0,1])
    plt.imshow(gray_bgr)
    plt.plot(vert_vanish_xs, vert_vanish_ys, 'b.')
    plt.plot(vert_vanish_pt[0], vert_vanish_pt[1], 'r.', markersize=16)  
    plt.plot(hori_vanish_xs, hori_vanish_ys, 'g.')
    plt.xlim(-10000, 10000), plt.ylim(-10000, 10000)
    plt.title('Vanishing Points')

    #Projective Transform Geometry Calculations Plot
    axptg = fig2.add_subplot(gs2[0,2])
    plt.imshow(gray_sample_box)
    plt.title('Projective Transform Geometry')

    #Projective Transform 
    axptg = fig2.add_subplot(gs2[1,0])
    plt.imshow(transed)
    plt.title('Projective Transform')

    #Update figure and Pause
    plt.draw()
    fig.canvas.manager.window.raise_()
    plt.pause(0.01)
    
    #Save all images as files
    if debug_outputs:
        
        # Sample lines 
        output_path = os.path.join(output_dir, image_root + '_lines' + images_ext)
        cv.imwrite(output_path, cv.cvtColor(gray_sample_lines, cv.COLOR_BGR2RGB))
        
        # Sample Boxes 
        output_path = os.path.join(output_dir, image_root + '_boxes' + images_ext)
        cv.imwrite(output_path, cv.cvtColor(gray_sample_box, cv.COLOR_BGR2RGB))
        
        # Debug figures 
        output_path = os.path.join(output_dir, image_root + '_debug_1' + images_ext)
        fig.savefig(output_path)
        output_path = os.path.join(output_dir, image_root + '_debug_2' + images_ext)
        fig2.savefig(output_path)
        
        #Close figures
        plt.close('all')

## Image Segmentation

In [5]:
%matplotlib 
# Define list of images in the specified directory (NEEDS EDGE HANDLING)
image_list = os.listdir(images_dir)
image_list = [image_name for image_name in image_list if image_name.endswith(images_ext)]

# Predefined global image processing parameters
resize_rows = 600    

#Processing steps
equalize_hist = False
equalize_method = 'global'
    
    
#iterate through images
image_it = 1
for image_name in image_list:
    
    #Log start time
    time_start = time.time()
    
    #Debug Outputs
    print(f'\nProcessing Image {image_name} ({image_it}):') 
    
    #Define image path
    image_root = os.path.splitext(image_name)[0]
    image_path = images_dir + '/' + image_name
    
    #Load image
    image_original = cv.imread(image_path)
    
    #Calculate basic parameters
    image_rows, image_cols, image_channels = image_original.shape;
    aspect_ratio = image_cols/image_rows
    image_pixels = image_cols * image_rows
    image_scalar = resize_rows/image_rows
    resize_cols = round(aspect_ratio * resize_rows)
    resize_pixels = resize_rows * resize_cols
    print(f'   Image Loading:')
    print(f'      Aspect Ratio: {aspect_ratio:.2f}')
    print(f'      Image Size is {image_rows} x {image_cols} ({image_pixels} px.)')
    print(f'      Image Resized to {resize_rows} x {resize_cols} ({resize_pixels} px.)')
    
    #Distortion correction - POSSIBLY IMPLEMENT - May cause more distortion than it corrects
    if correct_distortion:
        
        #Print update statement
        print('   Camera Distortion Correction:')
        
        if correct_distortion_mode == 'lensfun':                 #Corrects using lensfunpy, which uses the lensfun database
            
            #Error Handling
            try: 
                
                #Load camera and lens information
                lensfun_db = lensfunpy.Database()
                camera = lensfun_db.find_cameras(camera_mnfcr, camera_model)[0]
                lens = lensfun_db.find_lenses(camera, lens_mnfcr, lens_model)[0]

                #Correct distortion
                distortion_modifier = lensfunpy.Modifier(lens, camera.crop_factor, image_cols, image_rows)
                distortion_modifier.initialize(image_focal_length, image_aperture, approx_distance)
                undistorted_coords = distortion_modifier.apply_geometry_distortion()
                undistorted = cv.remap(image_original, undistorted_coords, None, cv.INTER_LANCZOS4)
                
                #Print update
                print(f'      Correction applied for {camera_model} with {lens_model}.')
                
            except Exception:
                
                print(Exception)
                
                #Print update
                print(f'      Correction failed. Invalid camera or lens inputs.')
            
        elif correct_distortion_mode == 'camera_calibration':     #Uses an existing calibration file from calibrate_camera utility
            
            x=y
            
        else:
            print('      Distortion Correction Failed. Check inputs')
            undistorted = image_original
    
    #Resize image to nominal size to increase computation speed(and adjust the anticipated aspect ratio as well)
    
    image = cv.resize(undistorted, (resize_cols, resize_rows))
        
    #NEED PREPROCESSING STEP WHICH ADJUSTS THE WHITE BALANCE SOMEHOW - WHEN IMAGING, WB should be constant
    
    #Convert to RGB and isolate channels
    rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    rgb_min = np.amin(rgb, axis=2)
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    red = rgb[:,:,0]
    gre = rgb[:,:,1]
    blu = rgb[:,:,2]
    
    #Calculate RGB statistics from the image 
    red_mean = np.mean(red)
    red_hist = cv.calcHist([rgb], [0], None, [256], [0, 256])
    gre_mean = np.mean(gre)
    gre_hist = cv.calcHist([gre], [0], None, [256], [0, 256])
    blu_mean = np.mean(blu)
    blu_hist = cv.calcHist([blu], [0], None, [256], [0, 256])
    print(f'   Image Statistics:')
    print(f'      RGB Means: red = {red_mean:.2f};'\
          f' blue = {blu_mean:.2f}; green = {gre_mean:.2f}')
    
    
    #Convert to HSV and isolate channels
    hsv = cv.cvtColor(rgb, cv.COLOR_RGB2HSV)
    hue = hsv[:,:,0]
    sat = hsv[:,:,1]
    val = hsv[:,:,2]
    
    #Calculate HSV statistics from the image
    #hue = hue.astype(np.float64)
    #hue[hue < 48] = hue + 48
    hue_mean = np.mean(hue)
    hue_hist = cv.calcHist([hue], [0], None, [256], [0, 256])
    sat_mean = np.mean(sat)
    sat_hist = cv.calcHist([sat], [0], None, [256], [0, 256])
    val_mean = np.mean(val)
    val_hist = cv.calcHist([val], [0], None, [256], [0, 256])
    print(f'      HSV Means: hue = {hue_mean:.2f};'\
          f' saturation = {sat_mean:.2f}; value = {val_mean:.2f}')

    #Detect if image is particularly dark 
    
    
    
    #Histogram Equalization
    if equalize_hist and (equalize_method == 'adaptive'):
        clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(9,9))
        rgb_min = clahe.apply(rgb_min)
        gray = clahe.apply(gray)
        hue = clahe.apply(hue)
    elif equalize_hist and (equalize_method == 'global'):
        rgb_min = cv.equalizeHist(rgb_min)
        gray = cv.equalizeHist(gray)
        hue = cv.equalizeHist(hue)
    
    
    #Iteratively refine detected edges in the images
    edges, edge_outputs = segment.edgeDetection(rgb, rgb_min, gray, hue, image_pixels, debug=debug)
    rgb_min_smooth = edge_outputs['rgb_min_smooth']
    gray_smooth = edge_outputs['gray_smooth']
    edges_rgb_min = edge_outputs['edges_rgb_min']
    edges_gray = edge_outputs['edges_gray']
    edges_hue = edge_outputs['edges_hue']
    
    #Detect prominent lines
    print('   Line Detection:')
    max_std = 1                                     #STD cutoff for line deviation
    min_line_length = 150
    max_line_gap = 10
    hough_threshold = 100
    hough_theta = np.pi/1080
    hough_rho = 1
    line_it = 0
    line_repeat = True
    while line_repeat & (line_it < 10):
    
        #Calculate lines with specified parameters
        lines, line_points, line_points_edge, line_angles = segment.lineDetection(edges, hough_rho, hough_theta, \
                                                                                  hough_threshold, min_line_length,\
                                                                                  max_line_gap, resize_rows, resize_cols, \
                                                                                  debug=debug)
        
        #Filter lines 
        vert_inds, vert_lines, vert_angles, hori_inds, hori_lines, hori_angles, reject_inds, reject_lines, line_outputs \
            = segment.filterLines(lines, line_points, line_angles, hough_theta, debug = True)
        
        #Extract additional outputs
        vert_angles_mean = line_outputs['vert_angles_mean']
        vert_angles_std = line_outputs['vert_angles_std']
        hori_angles_mean = line_outputs['hori_angles_mean']
        hori_angles_std = line_outputs['hori_angles_std']
        
        #Determine vanishing points
        vert_vanish_pt = line_outputs['vert_vanish_pt']
        vert_vanish_xs = line_outputs['vert_vanish_xs']   
        vert_vanish_ys = line_outputs['vert_vanish_ys']
        hori_vanish_xs = []
        hori_vanish_ys = []
        
        #Count number of final lines
        num_lines = len(lines)
        num_hori_lines = len(hori_lines)
        num_vert_lines = len(vert_lines)
        num_lines_filtered = num_lines -(num_hori_lines + num_vert_lines)
        
        #Determine if the line detection procedure should be repeated and strengthen or weaken parameters to achieve this
        max_ct = 250
        min_ct = 2      
        if (num_vert_lines < min_ct) or (num_hori_lines < min_ct):
            line_repeat = True
            min_line_length *= 0.9 
            max_line_gap *= 1.25 
            max_std *= 1.1
        elif (num_vert_lines > max_ct) or (num_hori_lines > max_ct):
            line_repeat = True
            min_line_length = 1.1 * min_line_length
            max_line_gap = 0.9 * max_line_gap
            max_std *= 0.75
        else:
            line_repeat = False
            
        #Output update
        print(f'      ({line_it}): {num_lines} Lines Detected - {num_lines_filtered} Filtered - '\
              f'{num_hori_lines} Horizontal ({hori_angles_mean:.2f} +- {hori_angles_std:.2f} deg.)',\
              f'- {num_vert_lines} Vertical ({vert_angles_mean:.2f} +- {vert_angles_std:.2f} deg.)')
        
        #Iterate iteration counter
        line_it = line_it + 1
        
        #Failure update
        if line_repeat and (line_it == 10):
            print('      Line Detection Failed - Image Rejected')
        

    #Plot lines on the image
    if debug:
        gray_bgr = cv.cvtColor(gray_smooth, cv.COLOR_GRAY2BGR)
        gray_bgr = segment.plotLines(gray_bgr, vert_lines, hori_lines, reject_lines)
        gray_bgr_extended = cv.cvtColor(gray_smooth, cv.COLOR_GRAY2BGR)
        gray_bgr_extended = segment.plotLines(gray, None, None, None, all_lines = line_points_edge)
    
    
    #Plot vanishing points and perspective lines
    #if debug:
    #    gray_vanish_pts = gray_bgr
    #    for v in range(0,len(vert_vanish_xs)):
    #        if not (math.isnan(vert_vanish_xs[v])) and not (math.isnan(vert_vanish_ys[v])):
    #            cv.circle(gray_vanish_pts, (int(vert_vanish_xs[v]), int(vert_vanish_ys[v])), 1, (0,128,255), thickness=-1)
    #    for h in range(0,len(hori_vanish_xs)):
    #        if not (math.isnan(hori_vanish_xs[h])) and not (math.isnan(hori_vanish_ys[h])):
    #            cv.circle(gray_vanish_pts, (int(hori_vanish_xs[h]), int(hori_vanish_ys[h])), 1, (0,255,128), thickness=-1)
            
    #Select 4 representative lines - 2 Horizontal and 2 Vertical
    #vert_lines_sort = np.argsort(vert_angles)
    #vert_lines = vert_lines[vert_lines_sort]
    #hori_lines_sort = np.argsort(hori_angles)
    #hori_lines = hori_lines[hori_lines_sort]

    #PASTE HERE
    
    #Calculate mean projective transform
    print('   Projective Transform:')
    transed, proj_trans, rot_ang, gray_sample_lines, gray_sample_box = segment.projectiveTransform(image_original, lines,\
                                                                                                   vert_inds, hori_inds,\
                                                                                                   gray_smooth, resize_rows,\
                                                                                                   image_scalar, hough_theta)
    
    
    
    #Save image
    transed = cv.cvtColor(transed, cv.COLOR_BGR2RGB)
    output_path = output_dir + image_root + '_output' + images_ext
    cv.imwrite(output_path, cv.cvtColor(transed, cv.COLOR_BGR2RGB))
    
    #Log end time and calculate total execution time
    time_end = time.time()
    time_execute = time_end - time_start
    print(f'   Processing time: {time_execute:0.4f} s')
    
    #Generate debug plots
    if debug:
        generateDebugPlots()
        
    #Iteration counter    
    image_it += 1
        
        
        
        
        

    

Using matplotlib backend: Qt5Agg

Processing Image DSC03003.JPG (1):
   Image Loading:
      Aspect Ratio: 1.50
      Image Size is 2832 x 4240 (12007680 px.)
      Image Resized to 600 x 898 (538800 px.)
   Camera Distortion Correction:
      Correction applied for NIKON D3S with Samyang T-S 24mm f/3.5 ED AS UMC.
   Image Statistics:
      RGB Means: red = 123.13; blue = 104.38; green = 74.86
      HSV Means: hue = 139.96; saturation = 147.05; value = 137.80
   Edge Detection:
      (0): Ratio = 0.61%
   Line Detection:


C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


      (0): 92 Lines Detected - 88 Filtered - 2 Horizontal (1.99 +- 0.92 deg.) - 2 Vertical (96.57 +- 5.05 deg.)
   Projective Transform:
      Perspective correction successful!
   Processing time: 0.9255 s

Processing Image DSC03013.JPG (2):
   Image Loading:
      Aspect Ratio: 1.50
      Image Size is 2832 x 4240 (12007680 px.)
      Image Resized to 600 x 898 (538800 px.)
   Camera Distortion Correction:
      Correction applied for NIKON D3S with Samyang T-S 24mm f/3.5 ED AS UMC.
   Image Statistics:
      RGB Means: red = 97.06; blue = 121.51; green = 60.91
      HSV Means: hue = 139.00; saturation = 149.41; value = 128.87
   Edge Detection:
      (0): Ratio = 0.41%
   Line Detection:
      (0): 77 Lines Detected - 73 Filtered - 2 Horizontal (0.66 +- 0.67 deg.) - 2 Vertical (93.20 +- 3.07 deg.)
   Projective Transform:
      Perspective correction successful!
   Processing time: 0.8687 s

Processing Image DSC03021.JPG (3):
   Image Loading:
      Aspect Ratio: 1.50
      Image Si

      (0): 195 Lines Detected - 191 Filtered - 2 Horizontal (6.40 +- 0.52 deg.) - 2 Vertical (97.23 +- 2.60 deg.)
   Projective Transform:
      Perspective correction successful!
   Processing time: 0.9495 s

Processing Image DSC03032.JPG (14):
   Image Loading:
      Aspect Ratio: 1.50
      Image Size is 2832 x 4240 (12007680 px.)
      Image Resized to 600 x 898 (538800 px.)
   Camera Distortion Correction:
      Correction applied for NIKON D3S with Samyang T-S 24mm f/3.5 ED AS UMC.
   Image Statistics:
      RGB Means: red = 103.84; blue = 102.40; green = 72.52
      HSV Means: hue = 134.79; saturation = 137.38; value = 121.42
   Edge Detection:
      (0): Ratio = 0.52%
   Line Detection:
      (0): 157 Lines Detected - 153 Filtered - 2 Horizontal (8.22 +- 0.53 deg.) - 2 Vertical (96.95 +- 1.56 deg.)
   Projective Transform:
      Perspective correction successful!
   Processing time: 1.0065 s

Processing Image DSC03033.JPG (15):
   Image Loading:
      Aspect Ratio: 1.50
      I

C:\Users\RSDG\Documents\GitHub\OpenUVF\core\utils\segmentation_utils.py:400: RuntimeWarning: Mean of empty slice
  mean_vanish_pt = [np.nanmean(valid_xs), np.nanmean(valid_ys)]


...

   Processing time: 0.7647 s

Processing Image DSC03970.JPG (236):
   Image Loading:
      Aspect Ratio: 1.50
      Image Size is 2832 x 4240 (12007680 px.)
      Image Resized to 600 x 898 (538800 px.)
   Camera Distortion Correction:
      Correction applied for NIKON D3S with Samyang T-S 24mm f/3.5 ED AS UMC.
   Image Statistics:
      RGB Means: red = 18.18; blue = 15.03; green = 11.45
      HSV Means: hue = 115.08; saturation = 165.55; value = 20.16
   Edge Detection:
      (0): Ratio = 0.09%   -    Gauss STD = 0.52; Canny Thresh = 41.68
      (1): Ratio = 0.31%
   Line Detection:
      (0): 64 Lines Detected - 60 Filtered - 2 Horizontal (-4.99 +- 9.72 deg.) - 2 Vertical (115.47 +- 6.06 deg.)
   Projective Transform:
      Perspective correction successful!
   Processing time: 0.7779 s

Processing Image DSC03971.JPG (237):
   Image Loading:
      Aspect Ratio: 1.50
      Image Size is 2832 x 4240 (12007680 px.)
      Image Resized to 600 x 898 (538800 px.)
   Camera Distortion Cor

      Perspective correction successful!
   Processing time: 0.7879 s

Processing Image DSC03981.JPG (247):
   Image Loading:
      Aspect Ratio: 1.50
      Image Size is 2832 x 4240 (12007680 px.)
      Image Resized to 600 x 898 (538800 px.)
   Camera Distortion Correction:
      Correction applied for NIKON D3S with Samyang T-S 24mm f/3.5 ED AS UMC.
   Image Statistics:
      RGB Means: red = 20.82; blue = 16.22; green = 12.27
      HSV Means: hue = 118.19; saturation = 166.59; value = 22.78
   Edge Detection:
      (0): Ratio = 0.09%   -    Gauss STD = 0.55; Canny Thresh = 43.00
      (1): Ratio = 0.29%
   Line Detection:
      (0): 48 Lines Detected - 44 Filtered - 2 Horizontal (-10.50 +- 13.17 deg.) - 2 Vertical (101.48 +- 14.95 deg.)
   Projective Transform:
      Perspective correction successful!
   Processing time: 0.7809 s

Processing Image DSC03982.JPG (248):
   Image Loading:
      Aspect Ratio: 1.50
      Image Size is 2832 x 4240 (12007680 px.)
      Image Resized to 600

   Processing time: 0.5226 s

Processing Image DSCF4599.JPG (258):
   Image Loading:
      Aspect Ratio: 1.50
      Image Size is 2000 x 3008 (6016000 px.)
      Image Resized to 600 x 902 (541200 px.)
   Camera Distortion Correction:
      Correction applied for NIKON D3S with Samyang T-S 24mm f/3.5 ED AS UMC.
   Image Statistics:
      RGB Means: red = 80.89; blue = 116.96; green = 54.21
      HSV Means: hue = 118.21; saturation = 148.19; value = 127.10
   Edge Detection:
      (0): Ratio = 0.62%
   Line Detection:
      (0): 57 Lines Detected - 53 Filtered - 2 Horizontal (-1.69 +- 0.91 deg.) - 2 Vertical (84.21 +- 19.40 deg.)
   Projective Transform:
      Perspective correction successful!
   Processing time: 0.5217 s

Processing Image DSCF4602.JPG (259):
   Image Loading:
      Aspect Ratio: 1.50
      Image Size is 2000 x 3008 (6016000 px.)
      Image Resized to 600 x 902 (541200 px.)
   Camera Distortion Correction:
      Correction applied for NIKON D3S with Samyang T-S 24mm f

      Correction applied for NIKON D3S with Samyang T-S 24mm f/3.5 ED AS UMC.
   Image Statistics:
      RGB Means: red = 69.45; blue = 55.97; green = 29.71
      HSV Means: hue = 144.83; saturation = 135.52; value = 74.46
   Edge Detection:
      (0): Ratio = 0.35%
   Line Detection:
      (0): 44 Lines Detected - 40 Filtered - 2 Horizontal (6.87 +- 17.36 deg.) - 2 Vertical (85.96 +- 23.04 deg.)
   Projective Transform:
      Perspective correction successful!
   Processing time: 0.4608 s

Processing Image DSCF4701.JPG (270):
   Image Loading:
      Aspect Ratio: 1.50
      Image Size is 2000 x 3008 (6016000 px.)
      Image Resized to 600 x 902 (541200 px.)
   Camera Distortion Correction:
      Correction applied for NIKON D3S with Samyang T-S 24mm f/3.5 ED AS UMC.
   Image Statistics:
      RGB Means: red = 79.28; blue = 60.54; green = 32.38
      HSV Means: hue = 150.17; saturation = 138.15; value = 82.91
   Edge Detection:
      (0): Ratio = 0.28%
   Line Detection:
      (0): 33

   Projective Transform:
      Perspective correction successful!
   Processing time: 0.5230 s

Processing Image DSCF4846.JPG (281):
   Image Loading:
      Aspect Ratio: 1.50
      Image Size is 2000 x 3008 (6016000 px.)
      Image Resized to 600 x 902 (541200 px.)
   Camera Distortion Correction:
      Correction applied for NIKON D3S with Samyang T-S 24mm f/3.5 ED AS UMC.
   Image Statistics:
      RGB Means: red = 82.06; blue = 73.07; green = 39.22
      HSV Means: hue = 142.57; saturation = 133.70; value = 91.45
   Edge Detection:
      (0): Ratio = 0.54%
   Line Detection:
      (0): 52 Lines Detected - 48 Filtered - 2 Horizontal (6.19 +- 3.45 deg.) - 2 Vertical (86.18 +- 21.69 deg.)
   Projective Transform:
      Perspective correction successful!
   Processing time: 0.5097 s

Processing Image DSCF4852.JPG (282):
   Image Loading:
      Aspect Ratio: 1.50
      Image Size is 2000 x 3008 (6016000 px.)
      Image Resized to 600 x 902 (541200 px.)
   Camera Distortion Correction: